In [1]:
import MySQLdb

## 连接数据库

In [2]:
#连接数据库,数据库中已经有一个测试数据库‘python_test’
con = MySQLdb.connect(host="localhost", user="root", passwd="lxm123456",db="python_test",port=3306)

在这里， 我们虽然拿到了python的数据库连接， 但是不能在这个对象上直接对数据库进行操作， 还需要获取对应的操作游标才能进行数据库的操作，所以还需要进行下面的操作：

In [3]:
cur = con.cursor()

## 创建表格

In [ ]:
cur.execute('create table stu_info (name char(128) not null default "",age tinyint(3) not null default 0, sex enum("man","femal") not null default "man") engine=innodb charset=utf8')
    #0L

cur.execute 返回执行的sql 影响的行数， 因为这里是创建数据库， 所以是0L行

但到这里还并没有真正执行了sql语句， 必须使用MySQLdb.commit才是真正执行完毕

In [10]:
con.commit()
#到这里， 我们的表格才算真正创建完成

## 更新表数据

同理， 往表中写数据， 也是一样的操作流程 :先execute，然后commit;
不过， 更新数据的execute 稍有不同， 往表中写入数据时， 执行execute 方法， 有两种方式，第二种更好
    
    （1） 一种是直接execute(sql)， 然后commit 完成， sql里是写入的sql 语句

In [11]:
cur.execute("insert into stu_info (name, age, sex) values ('Yi_Zhi_Yu',25,'man')")
con.commit()

这会直接写入表中，但还有另外一种方式.
        
    （2）execute 可以接受两个参数， 第一个参数是sql语句， 不过这个sql中的values的内容使用占位符%s表示，第二个参数是实际的写入的values列表， 如下：



In [12]:
cur.execute("insert into stu_info (name, age, sex) values (%s,%s,%s)", ("Tony",25, "man"))
con.commit()

这种方式与第一中方式相比， 更清晰一些， 安全性也更好， 能有效防止sql注入

另外， cursor还有一个executemany, 参数和execute一样， 不过第二个参数可以传递多列表值， 达到多次执行某个语句的效果

In [14]:
cur.executemany("insert into stu_info (name, age, sex) values (%s,%s,%s)",(("LiMei",26,"femal"),("YuanYuan",28,"femal")))
con.commit()
#这里实际上就是执行了两次插入操作

## 数据查询

In [15]:
#例如
cur.execute("select * from stu_info")
stus = cur.fetchall() 

In [16]:
stus

(('Yi_Zhi_Yu', 25, 'man'),
 ('Tony', 25, 'man'),
 ('LiMei', 26, 'femal'),
 ('YuanYuan', 28, 'femal'),
 ('LiMei', 26, 'femal'),
 ('YuanYuan', 28, 'femal'))

In [17]:
#tuple形式， 我们可以通过循环输出：
for stu in stus:
    print "name: %s; age: %d; sex: %s" %(stu[0], stu[1], stu[2])

name: Yi_Zhi_Yu; age: 25; sex: man
name: Tony; age: 25; sex: man
name: LiMei; age: 26; sex: femal
name: YuanYuan; age: 28; sex: femal
name: LiMei; age: 26; sex: femal
name: YuanYuan; age: 28; sex: femal


那上面的查询虽然得到了每行的数据， 但结果集中并没有字段名， 如果要返回字段名， 如下操作：

In [18]:
cur = con.cursor(cursorclass=MySQLdb.cursors.DictCursor)
cur.execute("select * from stu_info")
cur.fetchall()

({'age': 25, 'name': 'Yi_Zhi_Yu', 'sex': 'man'},
 {'age': 25, 'name': 'Tony', 'sex': 'man'},
 {'age': 26, 'name': 'LiMei', 'sex': 'femal'},
 {'age': 28, 'name': 'YuanYuan', 'sex': 'femal'},
 {'age': 26, 'name': 'LiMei', 'sex': 'femal'},
 {'age': 28, 'name': 'YuanYuan', 'sex': 'femal'})

每个元素都是一个dict, 以key-value的形式展示了每个字段和对应的值

## 总结